In [ ]:
# Version history:
# 2022-12-05: v6: added function set_seed
# 2022-12-04: v5: CatBoost and transformers are added
# 2022-12-03: v4: added random_state=42 to make_label_distribution_equal. Fixed f-string in "Unexpected model tag: {model_tag}". Metrics: 0.497 +- 0.002, 0.502 +- 0.003
# 2022-12-03: v4: added function make_label_distribution_equal, added np.std to final output
# 2022-12-03: v3: added colab section. Metrics: 0.507, 0.497

# ===== Part0 - env preparation =====

## System info

In [ ]:
# Print system id
!nvidia-smi
!hostname
!uname -a
!df -kh /tmp

Tue Dec 13 19:04:32 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   71C    P0    30W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
!python -V  # If version < 3.9 then some f-string features may not work

Python 3.8.16


## Mount drive (if required)

In [ ]:
TO_USE_COLAB = None
try:
    PATH_MOUNT = "/content/drive"
    from google.colab import drive
    drive.mount(PATH_MOUNT)
    TO_USE_COLAB = True
except:
    TO_USE_COLAB = False
TO_USE_COLAB

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


True

In [ ]:
import os
if TO_USE_COLAB:
    PATH_MAIN_DIR = f"{PATH_MOUNT}/MyDrive/_PR_ROOT/_2022/2022-11_NLP-Huawei_Final_project/stocktwits_finsentiment_analysis/notebooks"
    #PATH_MAIN_DIR = f"{PATH_MOUNT}/MyDrive/nlp_final_prj/"
else:
    PATH_MAIN_DIR = "."
assert os.path.isdir(PATH_MAIN_DIR)

In [ ]:
#%cd $PATH_MAIN_DIR  # Commented not to put there temp data
!pwd

/content


# ===== Part 1: prepare dataset =====

## Imports 

In [ ]:
import numpy as np
import os
import pandas as pd
import random
from sklearn.model_selection import train_test_split
import time
import torch

## Paths and settings

In [ ]:
# Files and folders

DIR_DATA_SRC = os.path.join(PATH_MAIN_DIR, r'../data/interim/040_output__nb010_v1')

#FNAMES = ['VIX_RmSW=0_RmRep=0_1y_top10.csv', 'VIX_RmSW=0_RmRep=0_1y_top10.csv' ]  # Loads in <1 sec
FNAMES = ['AMZN_RmSW=0_RmRep=0_1y.csv.gz', 'NFLX_RmSW=0_RmRep=0_1y.csv.gz', ]  # Loads in ~10-20 seconds
#FNAMES = ['AAPL_RmSW=0_RmRep=0_1y.csv.gz', ]  # Loads in 20-30 sec

assert os.path.isdir(DIR_DATA_SRC), f"Folder not found: {DIR_DATA_SRC}"
for f in FNAMES:
    assert os.path.isfile(os.path.join(DIR_DATA_SRC, f)), f"File not found: {f}"

In [ ]:
# Dataset preparation settings

DROP_RECORDS_BEFORE_DATE_INCLUSIVE = '2019-07-20'  # Last date in datasets is 2020-07-21
LABEL_GEN_STRATEGY = "d1_C=d1_O=0.5%=2cls"  # This string is a "key", see function XXX for explanations
COL_FEATURES = ['symbol', 'message', 'datetime', 'user', 'message_id', 'Date']  #, 'Time']
COL_LABEL = 'label'
COL_PCR = 'price_change_ratio'

TEST_SIZE = 0.15
TRAIN_SIZE = 1.0 - TEST_SIZE

## Defs
Here are "pure" functions.

In [ ]:
# More info: https://pytorch.org/docs/stable/notes/randomness.html
def init_seeds(seed=42):
    # Python and CPU-related entropy  
    random.seed(seed)      
    os.environ["PYTHONHASHSEED"] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    # torch.use_deterministic_algorithms(True)   # Raises a CUBLAS error on some cases
    # os.environ["CUBLAS_WORKSPACE_CONFIG"] = ":4096:8"  # Does not help for the error above

    # GPU-related entropy
    if torch.cuda.is_available():
        torch.cuda.manual_seed(seed)
        torch.cuda.manual_seed_all(seed) # gpu vars
        torch.backends.cudnn.benchmark = False  # See 
        torch.backends.cudnn.deterministic = True

In [ ]:
# Function for the "worker_init_fn" param of torch DataLoader
# More info: https://pytorch.org/docs/stable/notes/randomness.html
def seed_worker(worker_id):
    worker_seed = torch.initial_seed() % 2**32
    random.seed(worker_seed)
    np.random.seed(worker_seed)

In [ ]:
def print_df_details(df: pd.DataFrame):
    print("\nHead:\n", df.head())
    print("\nTail:\n", df.tail())
    print('\nInfo:')
    df.info()  # This method prints by itself
    print('\nDescribe:\n', df.describe(include='all'))  #, datetime_is_numeric=True)) - to suppress warnings   

In [ ]:
def load_pandas_file(file_path: str, verbose=True):
    # Prepare
    assert os.path.isfile(file_path), f"Cannot find file: '{file_path}', cur folder: '{os. getcwd()}'"    
    print("Loading data from: ", file_path)
        
    # Do the load
    start_time = time.time()
    df = pd.read_csv(file_path)
    print(f"Success. Shape: {df.shape}, elapsed seconds: {time.time() - start_time:.2f}")
    
    # Dump details if required
    if verbose:
        print_df_details(df)
    return df

In [ ]:
def merge_dfs(df_list: list, verbose=True) -> pd.DataFrame:
    if verbose:
        for df in df_list:
            print(df.shape, end=';')
    res_df = pd.concat(df_list, ignore_index=True)
    if verbose:
        print("->", res_df.shape)
    return res_df

In [ ]:
def drop_old_dates_inplace(df: pd.DataFrame, drop_date_inclusive: str, verbose=True) -> pd.DataFrame:
    assert isinstance(drop_date_inclusive, str)
    old_shape = df.shape
    df.drop(df[df['Date'] <= drop_date_inclusive].index, inplace = True)
    print(f"Old dates dropped. Shape before: {old_shape}, after: {df.shape}")
    if verbose:
        print_df_details(df)

In [ ]:
def get_label(ch):
  if ch > 0.5:
    return 1
  elif ch < -0.5:
    return -1
  else:
    return 0


def generate_labels_and_pcr_list(df: pd.DataFrame, strategy_str: str) -> list:
    # price_change_ratio = pcr 
    if strategy_str == "d1_C=d1_O=0.5%=2cls":
        assert (df['d1_O'] > 0.0).all()  # Prices must be > 0
        assert (df['d1_C'] > 0.0).all()  # Prices must be > 0
        rel_change_perc = (df['d1_C'] / df['d1_O'] - 1.0) * 100.0
        # Convert from percentages to labels -1, 0, 1
        res_series = rel_change_perc.apply(get_label)
    else:
        assert False, "Unexpeced strategy_str"
    return res_series.to_list(), rel_change_perc.to_list()     

In [ ]:
def do_feature_selection(df: pd.DataFrame):
    res_df = df[COL_FEATURES]
    print(f"Selected cols: {res_df.columns}")
    return res_df.copy()

In [ ]:
def do_label_transformation(df: pd.DataFrame):
    temp_df = df.drop(df[df[COL_LABEL] == 0].index, inplace= False).copy()
    temp_df[COL_LABEL].replace({-1:0}, inplace = True)
    return temp_df

In [ ]:
def calc_real_profit_perc(y_pred, pcr_list) -> float:
    return np.NaN  # TODO: This function is not correct, as it's necessary to aggregate predictions by date and ticker

    profit_ratio = 1.0
    assert len(y_pred) == len(pcr_list), f"{len(y_pred)}, {len(pcr_list)}"
    for i, (pred, pcr) in enumerate(zip(y_pred, pcr_list)):
        price_ratio = (pcr / 100.0 + 1.0)  # Convert from percents [-5% .. 5%] -> [-0.05 .. 0.05] -> [0.95 .. 1.05]
        assert 0.0 < price_ratio < np.inf, f"{i}, {price_ratio}" 
        if pred == 1:
            # Long
            profit_ratio *= price_ratio
        elif pred == 0:
            # Short
            profit_ratio /= price_ratio
        else:
            assert False, "Unexpected label"
    return (profit_ratio - 1.0) * 100.0  # Profit in percents (0% - nothing changed)

In [ ]:
def calc_hash_for_seq(values, hash_len=6):
    assert isinstance(values, (list, np.ndarray, pd.Series))
    h = hash(tuple(values))
    return str(h)[-hash_len:]

# Small unit tests
print(calc_hash_for_seq([1, 2, 3]))
print(calc_hash_for_seq(np.array([1, 2, 3])))
print(calc_hash_for_seq(pd.Series([1, 2, 3])))

497451
497451
497451


In [ ]:
def make_label_distribution_equal(df: pd.DataFrame, random_state=42) -> pd.DataFrame:
    
    counts = df.label.value_counts()
    assert len(counts == 2)  # We expect only labels 0 and 1

    bigger_label = 0 if counts[0] > counts[1] else 1
    diff = abs(counts[0] - counts[1])

    res_df = df.drop(index=df[df.label == bigger_label].sample(n = diff, replace=False, random_state=random_state).index)
    return res_df

## Do prepare datasets

In [ ]:
#init_seeds(42)  # May be useful if torch DataLoader is used, etc.  TODO: think if required here

In [ ]:
# Load raw data, dropping old dates
df_list = []
for fname in FNAMES:
    full_name = os.path.join(DIR_DATA_SRC, fname)
    assert os.path.isfile(full_name), full_name
    df_temp = load_pandas_file(full_name, verbose=False)
    drop_old_dates_inplace(df_temp, DROP_RECORDS_BEFORE_DATE_INCLUSIVE, verbose=False)
    df_list.append(df_temp)

Loading data from:  /content/drive/MyDrive/_PR_ROOT/_2022/2022-11_NLP-Huawei_Final_project/stocktwits_finsentiment_analysis/notebooks/../data/interim/040_output__nb010_v1/AMZN_RmSW=0_RmRep=0_1y.csv.gz
Success. Shape: (450379, 52), elapsed seconds: 4.93
Old dates dropped. Shape before: (450379, 52), after: (100280, 52)
Loading data from:  /content/drive/MyDrive/_PR_ROOT/_2022/2022-11_NLP-Huawei_Final_project/stocktwits_finsentiment_analysis/notebooks/../data/interim/040_output__nb010_v1/NFLX_RmSW=0_RmRep=0_1y.csv.gz
Success. Shape: (666002, 52), elapsed seconds: 6.90
Old dates dropped. Shape before: (666002, 52), after: (110002, 52)


In [ ]:
# Concat loaded parts to one dataframe
df_raw = merge_dfs(df_list)

(100280, 52);(110002, 52);-> (210282, 52)


In [ ]:
# Choose columns for final dataset
df_final = do_feature_selection(df_raw)

Selected cols: Index(['symbol', 'message', 'datetime', 'user', 'message_id', 'Date'], dtype='object')


In [ ]:
# Append the target column
labels, pcr_list = generate_labels_and_pcr_list(df_raw, strategy_str=LABEL_GEN_STRATEGY)
df_final[COL_LABEL] = labels
df_final[COL_PCR] = pcr_list

In [ ]:
# Drop labels for neutral class
df_final = do_label_transformation(df_final)

In [ ]:
# print_df_details(df_final)

In [ ]:
df_final[COL_LABEL].value_counts()

0    64504
1    56793
Name: label, dtype: int64

In [ ]:
# Making labels distribution equal
df_final = make_label_distribution_equal(df_final)
df_final[COL_LABEL].value_counts()

0    56793
1    56793
Name: label, dtype: int64

In [ ]:
df_final

,symbol,message,datetime,user,message_id,Date,label,price_change_ratio
0,AMZN,amzn believe we see a msft reaction,2020-07-22 22:36:53+00:00,1138814,230085465,2020-07-22,0,-3.602092
1,AMZN,ba tsla amzn ge googl new ceo of boeing david ...,2020-07-22 22:36:44+00:00,3433309,230085430,2020-07-22,0,-3.602092
2,AMZN,amzn should be back to 3300 after earning mar...,2020-07-22 22:33:29+00:00,1791337,230084558,2020-07-22,0,-3.602092
3,AMZN,amzn aapl anti trust congressional hearings st...,2020-07-22 22:33:09+00:00,767238,230084470,2020-07-22,0,-3.602092
4,AMZN,tsla musk wants tsla to be quot slightly profi...,2020-07-22 22:29:24+00:00,909664,230083378,2020-07-22,0,-3.602092
...,...,...,...,...,...,...,...,...
210152,NFLX,nflx has one of the better altman z scores in ...,2019-07-22 01:03:58+00:00,47688,171262286,2019-07-22,0,-1.329309
210155,NFLX,just charted nflx gld st ne in the snm room gt...,2019-07-22 00:40:18+00:00,186026,171261315,2019-07-22,0,-1.329309
210156,NFLX,nflx pivot point 318 39 hourly interesting t...,2019-07-22 00:36:45+00:00,2121786,171261168,2019-07-22,0,-1.329309
210157,NFLX,spy fb aapl amzn nflx googl amzn is a beast so...,2019-07-22 00:28:08+00:00,1323307,171260827,2019-07-22,0,-1.329309


# ===== Part 2: Model execution and scoring =====

## Imports (part 2)

In [ ]:
import pickle
from typing import Optional

# Baseline-related imports
from sklearn.dummy import DummyClassifier
from sklearn.metrics import classification_report, f1_score, accuracy_score
from sklearn.metrics import confusion_matrix

## Defs (part 2)

In [ ]:
def train_model_and_get_predictions__sklearn_classifier(model_tag: str, 
    X_train: pd.DataFrame, y_train: pd.Series, X_test: pd.Series, seed: int) -> pd.Series:

    # Initial checks
    assert COL_PCR not in X_train.columns  # To avoid data leaks
    
    # Create the model with specified seed
    if model_tag == "dummy__most_frequent":
        model = DummyClassifier(strategy="most_frequent", random_state=seed)
    elif model_tag == "dummy__uniform":
        model = DummyClassifier(strategy="uniform", random_state=seed)
    else:
        assert False, f"Unexpected model tag: {model_tag}"
    
    # Train the model    
    model.fit(X_train, y_train)
    
    # Get predictions
    y_pred = model.predict(X_test)
    
    return y_pred    

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB, GaussianNB

def train_model_and_get_predictions__NB_classifier(model_tag: str, 
    X_train: pd.DataFrame, y_train: pd.Series, X_test: pd.Series, seed: int, out_model_file: Optional[str]=None) -> pd.Series:

    # Initial checks
    assert COL_PCR not in X_train.columns  # To avoid data leaks
    
    # Create the vectorizer and model with specified params
    if model_tag == "NaiveBayes_words_ng1-3_alhpa0.1":
        tf_idf = TfidfVectorizer(use_idf=True, ngram_range=(1,3), analyzer='word')
        model = MultinomialNB(alpha=0.1)
    elif model_tag == "NaiveBayes_words_ng2-2_alhpa0.1":
        tf_idf = TfidfVectorizer(use_idf=True, ngram_range=(2,2), analyzer='word')
        model = MultinomialNB(alpha=0.1)        
    else:
        assert False, f"Unexpected model tag: {model_tag}"
    
    # Prepare tf-idf features (!huge sparse matrix)
    train_features = tf_idf.fit_transform(X_train.message)
    test_features = tf_idf.transform(X_test.message)

    # Train the model    
    model.fit(train_features, y_train)

    # Save the model if required (will be overwritten if the file exists)
    if out_model_file is not None:
        pickle.dump(model, open(f"{out_model_file}_NB.pkl", 'wb'))
        pickle.dump(tf_idf, open(f"{out_model_file}_TFIDF.pkl", 'wb'))
        print("Model and tf-idf saved to output files.")
    
    # Get predictions
    y_pred = model.predict(test_features)
    
    return y_pred   

In [ ]:
!pip install catboost --quiet

from catboost import CatBoostClassifier

def train_model_and_get_predictions__catboost(model_tag: str, 
    X_train: pd.DataFrame, y_train: pd.Series, X_test: pd.Series, seed: int) -> pd.Series:

    # Initial checks
    assert COL_PCR not in X_train.columns  # To avoid data leaks
    
    model = CatBoostClassifier(
                                n_estimators=300,
                                max_depth = 8,
                                task_type = 'GPU',
                                verbose = 0
                                )
    
    # Train the model    
    model.fit(X_train.loc[:,['message']], y_train, text_features=['message'])
    
    # Get predictions
    y_pred = model.predict(X_test.loc[:,['message']])
    
    return y_pred   

     |████████████████████████████████| 76.6 MB 1.2 MB/s 


In [ ]:
!pip install wandb --quiet
!pip install simpletransformers --quiet

import torch
from simpletransformers.classification import ClassificationModel, ClassificationArgs

def train_model_and_get_predictions__transformer(model_tag: str, 
    X_train: pd.DataFrame, y_train: pd.Series, X_test: pd.Series, seed: int) -> pd.Series:

    # Initial checks
    assert COL_PCR not in X_train.columns  # To avoid data leaks
    
    is_cuda = True if torch.cuda.is_available() else False

    model_args = ClassificationArgs()
    model_args.num_train_epochs = 2
    model_args.regression = False
    model_args.use_multiprocessing=is_cuda
    model_args.use_multiprocessing_for_evaluation=is_cuda
    model_args.overwrite_output_dir=True
    model_args.train_batch_size = 128

    model = ClassificationModel(
                                "roberta",
                                "distilroberta-base",
                                num_labels=2,
                                use_cuda=is_cuda,
                                args=model_args
                                )
    
    # Train the model    
    tmp_df = pd.concat([X_train['message'], y_train], axis=1)
    tmp_df.columns = ["text", "labels"]
    model.train_model(tmp_df)
    
    # Get predictions
    y_pred, _ = model.predict(list(X_test['message']))
    
    return y_pred   

     |████████████████████████████████| 1.9 MB 29.2 MB/s 
     |████████████████████████████████| 168 kB 63.8 MB/s 
     |████████████████████████████████| 182 kB 56.0 MB/s 
     |████████████████████████████████| 62 kB 1.3 MB/s 
     |████████████████████████████████| 168 kB 73.0 MB/s 
     |████████████████████████████████| 166 kB 70.6 MB/s 
     |████████████████████████████████| 166 kB 58.9 MB/s 
     |████████████████████████████████| 162 kB 71.4 MB/s 
     |████████████████████████████████| 162 kB 75.1 MB/s 
     |████████████████████████████████| 158 kB 76.0 MB/s 
     |████████████████████████████████| 157 kB 68.6 MB/s 
     |████████████████████████████████| 157 kB 72.0 MB/s 
     |████████████████████████████████| 157 kB 69.4 MB/s 
     |████████████████████████████████| 157 kB 58.9 MB/s 
     |████████████████████████████████| 157 kB 75.9 MB/s 
     |████████████████████████████████| 157 kB 76.4 MB/s 
     |████████████████████████████████| 157 kB 77.4 MB/s 
     |██████████

## Main function (cycle for several seeds)

In [ ]:
# Launch split-train-predict-metrics cycle for several seeds
def get_model_score_distribution(model_tag: str, df: pd.DataFrame, launch_cnt: int = 5, verbose=True):
    result = []
    print("Legend: seed; X_train shape; X_test_shape; y_train shape,hash,sum; y_test shape,hash,sum")
    for seed in range(42, 42 + launch_cnt):
        X_train, X_test, y_train, y_test = train_test_split(
            df[COL_FEATURES + [COL_PCR]], df[COL_LABEL],
            # stratify=df[COL_LABEL], # Note: stratification leads to the same test set (though shuffled)
            shuffle=True,
            random_state = seed, 
            test_size = TEST_SIZE
        )
        # Note: equal hash means binary equality, equal sum means the same rows but shuffled
        print(f"After split: {seed}, {X_train.shape}; {X_test.shape}; {y_train.shape},{calc_hash_for_seq(y_train)},{sum(y_train)};"
              + f" {y_test.shape},{calc_hash_for_seq(y_test)},{sum(y_test)}")

        # Separate price_change_ratio from the data
        pcr_train = X_train[COL_PCR]; X_train.drop(COL_PCR, axis=1, inplace=True)
        pcr_test = X_test[COL_PCR]; X_test.drop(COL_PCR, axis=1, inplace=True)

        # Launch model-specific method
        y_pred = None
        if model_tag.startswith('dummy_'):
            y_pred = train_model_and_get_predictions__sklearn_classifier(model_tag, X_train, y_train, X_test, seed)
        elif model_tag.startswith('NaiveBayes_'):
            out_model_path = f"naive_bayes_model__seed{seed}"
            y_pred = train_model_and_get_predictions__NB_classifier(model_tag, X_train, y_train, X_test, seed, out_model_path)
        elif model_tag =='catboost':
            y_pred = train_model_and_get_predictions__catboost(model_tag, X_train, y_train, X_test, seed)
        elif model_tag =='transformer':
            y_pred = train_model_and_get_predictions__transformer(model_tag, X_train, y_train, X_test, seed)
        else:
            assert False, f"Unexpected model tag: {model_tag}"

        # Calc score
        score1 = accuracy_score(y_test, y_pred)
        #score2 = calc_real_profit_perc(y_pred, pcr_test)
        #score3 = calc_real_profit_perc(y_train[:100], pcr_train[:100])
        #result.append(f"{score1:.5f}, {score2:.2f}%, {score3:.2f}%")
        result.append(score1)
            
        if verbose:
            print(confusion_matrix(y_test, y_pred))
            print(classification_report(y_test, y_pred, digits=3))
                    
    return result

## Launch the model training/estimation

In [ ]:
model_tag = "dummy__most_frequent"
results = get_model_score_distribution(model_tag, df_final, verbose=False)
print("Sorted results (accuracy):", sorted(results))
print(f"Mean accuracy: {np.mean(results):.3f} +- {np.std(results):.3f}")

Legend: seed; X_train shape; X_test_shape; y_train shape,hash,sum; y_test shape,hash,sum
After split: 42, (96548, 7); (17038, 7); (96548,),077111,48332; (17038,),802379,8461
After split: 43, (96548, 7); (17038, 7); (96548,),325290,48367; (17038,),784870,8426
After split: 44, (96548, 7); (17038, 7); (96548,),158103,48299; (17038,),603758,8494
After split: 45, (96548, 7); (17038, 7); (96548,),225964,48292; (17038,),863571,8501
After split: 46, (96548, 7); (17038, 7); (96548,),678696,48321; (17038,),537076,8472
Sorted results (accuracy): [0.4945416128653598, 0.4965958445826975, 0.49724146026528937, 0.49853269163047303, 0.4989435379739406]
Mean accuracy: 0.497 +- 0.002


In [ ]:
model_tag = "dummy__uniform"
results = get_model_score_distribution(model_tag, df_final, launch_cnt = 10, verbose=False)
print("Sorted results (accuracy):", sorted(results))
print(f"Mean accuracy: {np.mean(results):.3f} +- {np.std(results):.3f}")

In [ ]:
assert False  # All models below are already saved to files

# Train and save individual models

In [ ]:
model_tag = "NaiveBayes_words_ng1-3_alhpa0.1"  # Mean accuracy: 0.568 +- 0.002
#model_tag = "NaiveBayes_words_ng2-2_alhpa0.1"  # Mean accuracy: 0.559 +- 0.002
results = get_model_score_distribution(model_tag, df_final, launch_cnt = 2, verbose=False)
print("Sorted results (accuracy):", sorted(results))
print(f"Mean accuracy: {np.mean(results):.3f} +- {np.std(results):.3f}")

In [ ]:
! cp /content/naive_bayes_model__seed42_*.pkl $PATH_MAIN_DIR/../models/
! cp /content/naive_bayes_model__seed43_*.pkl $PATH_MAIN_DIR/../models/

In [ ]:
model_tag = "catboost"
results = get_model_score_distribution(model_tag, df_final, launch_cnt = 5, verbose=False)
print("Sorted results (accuracy):", sorted(results))
print(f"Mean accuracy: {np.mean(results):.3f} +- {np.std(results):.3f}")

In [ ]:
model_tag = "transformer"
results = get_model_score_distribution(model_tag, df_final, launch_cnt = 5, verbose=False)
print("Sorted results (accuracy):", sorted(results))
print(f"Mean accuracy: {np.mean(results):.3f} +- {np.std(results):.3f}")

# Model blending

## Make test set for specific seed

In [ ]:
BLEND_SEED = 42
print("Legend: seed; X_test_shape; hash,sum; y_test shape,hash,sum")

_, X_test, _, y_test = train_test_split(
    df_final[COL_FEATURES + [COL_PCR]], df_final[COL_LABEL],
    # stratify=df[COL_LABEL], # Note: stratification leads to the same test set (though shuffled)
    shuffle=True,
    random_state = BLEND_SEED, 
    test_size = TEST_SIZE
)
# Note: equal hash means binary equality, equal sum means the same rows but shuffled
print(f"After split: {BLEND_SEED} {X_test.shape}; "
      + f" {y_test.shape},{calc_hash_for_seq(y_test)},{sum(y_test)}")

Legend: seed; X_test_shape; hash,sum; y_test shape,hash,sum
After split: 42 (17038, 7);  (17038,),802379,8461


## Load the models (NB, CB, TR)

In [ ]:
# Load NB model
! cp $PATH_MAIN_DIR/../models/naive_bayes_model__seed42_*.pkl /content/ 

In [ ]:
model_nb = pickle.load(open('naive_bayes_model__seed42_NB.pkl', 'rb'))
model_tfidf = pickle.load(open('naive_bayes_model__seed42_TFIDF.pkl', 'rb'))

In [ ]:
features = model_tfidf.transform(X_test.message)
score_nb = model_nb.score(features, y_test)
assert np.isclose(score_nb, 0.572, atol=0.001)

In [ ]:
# Get probabilities
probs_nb = model_nb.predict_proba(features)[:, 1]
probs_nb

array([0.0419392 , 0.3036071 , 0.51478722, ..., 0.66667132, 0.31633421,
       0.76179808])

In [ ]:
# Load CB model
! cp $PATH_MAIN_DIR/../models/catboost_model_0559.cbm /content/ 

In [ ]:
from catboost import CatBoostClassifier
model_cb = CatBoostClassifier()
model_cb.load_model("catboost_model_0559.cbm")

In [ ]:
score_cb = model_cb.score(X_test.loc[:,['message']], y_test)
assert np.isclose(score_cb, 0.559, atol=0.001)

In [ ]:
# Get probabilities
probs_cb = model_cb.predict_proba(X_test.loc[:,['message']])[:, 1]
probs_cb

array([0.50181991, 0.45940911, 0.49781323, ..., 0.3711765 , 0.45208126,
       0.48286085])

In [ ]:
# Load transformer
! cp $PATH_MAIN_DIR/../models/roberta_model_0548.zip /content/ 

In [ ]:
! unzip roberta_model_0548.zip

Archive:  roberta_model_0548.zip
   creating: content/roberta_model/checkpoint-2265-epoch-3/
  inflating: content/roberta_model/checkpoint-2265-epoch-3/merges.txt  
  inflating: content/roberta_model/checkpoint-2265-epoch-3/tokenizer.json  
  inflating: content/roberta_model/checkpoint-2265-epoch-3/config.json  
  inflating: content/roberta_model/checkpoint-2265-epoch-3/optimizer.pt  
  inflating: content/roberta_model/checkpoint-2265-epoch-3/model_args.json  
  inflating: content/roberta_model/checkpoint-2265-epoch-3/vocab.json  
  inflating: content/roberta_model/checkpoint-2265-epoch-3/pytorch_model.bin  
  inflating: content/roberta_model/checkpoint-2265-epoch-3/training_args.bin  
  inflating: content/roberta_model/checkpoint-2265-epoch-3/tokenizer_config.json  
  inflating: content/roberta_model/checkpoint-2265-epoch-3/special_tokens_map.json  
  inflating: content/roberta_model/checkpoint-2265-epoch-3/scheduler.pt  


In [ ]:
!pip install wandb --quiet
!pip install simpletransformers --quiet
from simpletransformers.classification import ClassificationModel, ClassificationArgs

In [ ]:
is_cuda = True if torch.cuda.is_available() else False

model_args = ClassificationArgs()
model_args.num_train_epochs = 2
model_args.regression = False
model_args.use_multiprocessing=is_cuda
model_args.use_multiprocessing_for_evaluation=is_cuda
model_args.overwrite_output_dir=True
model_args.train_batch_size = 128

model_tr = ClassificationModel(
                            "roberta",
                            "content/roberta_model/checkpoint-2265-epoch-3/", #"distilroberta-base",
                            num_labels=2,
                            use_cuda=is_cuda,
                            args=model_args
                            )


In [ ]:
# Get predictions
y_pred_tr, y_probs_tr = model_tr.predict(list(X_test['message']))


  0%|          | 0/17038 [00:00<?, ?it/s]

  0%|          | 0/2130 [00:00<?, ?it/s]

In [ ]:
score_tr = accuracy_score(y_pred_tr, y_test)
assert np.isclose(score_tr, 0.548, atol=0.001)

In [ ]:
from scipy.special import softmax

In [ ]:
probs_tr = softmax(y_probs_tr, axis=1)[:, 1]
probs_tr

array([0.53479474, 0.33807712, 0.58064931, ..., 0.45675808, 0.43467126,
       0.58020339])

## Do blending for NB+CB, NB+TR, NB+CB+TR

In [ ]:
for p in np.arange(0, 1+1e-6, 0.1):
    q = 1.0 - p
    probs = (p * probs_nb + q * probs_cb) >= 0.5
    y_pred = probs * 1  # Bools to ints
    score = accuracy_score(y_test, y_pred)
    print(f"{p:.2f}, {score:.4f}") 

0.00, 0.5588
0.10, 0.5750
0.20, 0.5785
0.30, 0.5784
0.40, 0.5775
0.50, 0.5769
0.60, 0.5763
0.70, 0.5758
0.80, 0.5758
0.90, 0.5744
1.00, 0.5724


In [ ]:
for p in np.arange(0, 1+1e-6, 0.1):
    q = 1.0 - p
    probs = (p * probs_nb + q * probs_tr) >= 0.5
    y_pred = probs * 1  # Bools to ints
    score = accuracy_score(y_test, y_pred)
    print(f"{p:.2f}, {score:.4f}") 

0.00, 0.5475
0.10, 0.5603
0.20, 0.5704
0.30, 0.5765
0.40, 0.5775
0.50, 0.5760
0.60, 0.5760
0.70, 0.5747
0.80, 0.5732
0.90, 0.5742
1.00, 0.5724


In [ ]:
# 0 0 100
# 0 50 50
# 0 100 0
# 50 0 50
# 50 50 0
# 100 0 0

max_score = 0
max_pqz = ()
for p in range(0, 101, 10):
    for q in range(0, 101, 10):
      z = 100-p-q
      if p < 0 or q <0 or z < 0:
          continue
      #print(p, q, z, 100-p-q-z)

      pp = p / 100.0
      qq = q / 100.0
      zz = z / 100.0
      probs = (pp * probs_nb + qq * probs_cb + zz * probs_tr) >= 0.5
      y_pred = probs * 1  # Bools to ints
      score = accuracy_score(y_test, y_pred)
      if max_score < score:
          max_score = score
          max_pqz = (pp, qq, zz)
      print(f"{pp:.2f}, {qq:.2f}, {zz:.2f}: {score:.4f}, {max_score:.5f}") 

      # q = 1.0 - p
      # probs = (p * probs_nb + q * probs_cb) >= 0.5
      # y_pred = probs * 1  # Bools to ints
      # score = accuracy_score(y_test, y_pred)
      # print(p, score)

print(f"Best: {max_score:.5f}, {max_pqz}") 

0.00, 0.00, 1.00: 0.5475, 0.54754
0.00, 0.10, 0.90: 0.5483, 0.54830
0.00, 0.20, 0.80: 0.5512, 0.55124
0.00, 0.30, 0.70: 0.5550, 0.55499
0.00, 0.40, 0.60: 0.5561, 0.55605
0.00, 0.50, 0.50: 0.5586, 0.55857
0.00, 0.60, 0.40: 0.5626, 0.56257
0.00, 0.70, 0.30: 0.5654, 0.56544
0.00, 0.80, 0.20: 0.5659, 0.56591
0.00, 0.90, 0.10: 0.5601, 0.56591
0.00, 1.00, 0.00: 0.5588, 0.56591
0.10, 0.00, 0.90: 0.5603, 0.56591
0.10, 0.10, 0.80: 0.5643, 0.56591
0.10, 0.20, 0.70: 0.5664, 0.56644
0.10, 0.30, 0.60: 0.5684, 0.56838
0.10, 0.40, 0.50: 0.5714, 0.57137
0.10, 0.50, 0.40: 0.5732, 0.57319
0.10, 0.60, 0.30: 0.5747, 0.57466
0.10, 0.70, 0.20: 0.5762, 0.57624
0.10, 0.80, 0.10: 0.5762, 0.57624
0.10, 0.90, 0.00: 0.5750, 0.57624
0.20, 0.00, 0.80: 0.5704, 0.57624
0.20, 0.10, 0.70: 0.5724, 0.57624
0.20, 0.20, 0.60: 0.5753, 0.57624
0.20, 0.30, 0.50: 0.5772, 0.57724
0.20, 0.40, 0.40: 0.5795, 0.57953
0.20, 0.50, 0.30: 0.5783, 0.57953
0.20, 0.60, 0.20: 0.5791, 0.57953
0.20, 0.70, 0.10: 0.5778, 0.57953
0.20, 0.80, 0.